In [1]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np
import datetime
from importlib import reload
from sklearn.pipeline import FeatureUnion
import TSManager as ts

# Split training and testing dataset

In [24]:
reload(ts)
mgr = ts.TSManager()
mgr.load()

Loading CAPE
Loading DP
Loading SPX
Loading OIL
Loading CPI
Loading IC
Loading BDI
Loading PCR
Loading SI
Loading BM
Loading TERM
Loading DEF
Loading PE
Loading VRP
Loading PCAPrice
Loading BY
Loading SIM
Loading NOS
Loading MA
Loading CAY


### Train date: 1990,06,08-2015,05,04

In [43]:
data=mgr.data
last_train_date=datetime.datetime(2015, 5, 4)
train=data[data.index<=last_train_date]
test=data[data.index>last_train_date]

In [44]:
train.head()

,CAPE,DP,SPX,OIL,CPI,IC,BDI,PCR,SI,BM,TERM,DEF,PE,VRP,PCAPrice,BY,SIM,NOS,MA,CAY
AsOfDate,,,,,,,,,,,,,,,,,,,,
1990-06-08,8.978524,0.063979,-0.012226,NaN,0.046737,NaN,1278.0,-1.622923,NaN,NaN,0.50,0.96,7.524271,7.197727,3.616905,1.007418,93.0,0.016941,1,-0.013576
1990-06-11,9.051612,0.063463,0.008140,NaN,0.046737,NaN,1264.0,-1.615366,NaN,NaN,0.49,0.96,7.597792,5.607431,3.583706,1.009642,92.0,0.016941,1,-0.013576
1990-06-12,9.167251,0.062662,0.012775,NaN,0.046737,NaN,1254.0,-1.618868,NaN,NaN,0.49,0.96,7.699009,6.403252,3.532720,1.009580,91.0,0.016941,1,-0.013576
1990-06-13,9.133460,0.062894,-0.003686,NaN,0.046737,NaN,1243.0,-1.625316,NaN,NaN,0.47,0.96,7.674771,7.793147,3.547245,1.000056,90.0,0.016941,1,-0.013576
1990-06-14,9.083400,0.063241,-0.005481,NaN,0.046737,NaN,1232.0,-1.620454,NaN,NaN,0.46,0.96,7.636994,7.665179,3.569002,0.997689,89.0,0.016941,1,-0.013576


In [45]:
#future return
train.loc[:,'R_1M']=train['SPX'].index.map(
lambda x: (1+train.truncate(before=x + pd.Timedelta(1,unit='d'), after=x + pd.Timedelta(30,unit='d'))['SPX']).prod()-1)
train.loc[:,'R_3M']=train['SPX'].index.map(
lambda x: (1+train.truncate(before=x + pd.Timedelta(1,unit='d'), after=x + pd.Timedelta(91,unit='d'))['SPX']).prod()-1)
train.loc[:,'R_6M']=train['SPX'].index.map(
lambda x: (1+train.truncate(before=x + pd.Timedelta(1,unit='d'), after=x + pd.Timedelta(182,unit='d'))['SPX']).prod()-1)
train.loc[:,'R_12M']=train['SPX'].index.map(
lambda x: (1+train.truncate(before=x + pd.Timedelta(1,unit='d'), after=x + pd.Timedelta(365,unit='d'))['SPX']).prod()-1)

C:\Users\xjl05\AppData\Local\conda\conda\envs\python353\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [46]:
train.loc[last_train_date-pd.Timedelta(30,unit='d'):,'R_1M']=np.nan
train.loc[last_train_date-pd.Timedelta(91,unit='d'):,'R_3M']=np.nan
train.loc[last_train_date-pd.Timedelta(182,unit='d'):,'R_6M']=np.nan
train.loc[last_train_date-pd.Timedelta(365,unit='d'):,'R_12M']=np.nan

# Prepocess

In [47]:
#pipelines to select columns
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]
#de-median
class DeMedian(BaseEstimator, TransformerMixin):
    def __init__(self): # no *args or **kargs
        self.median=0
    def fit(self, X):
        self.median=X.median()
        return self  # nothing else to do
    def transform(self, X):
        return X-self.median
#take log
class Log(BaseEstimator, TransformerMixin):
    def __init__(self): 
        return
    def fit(self, X):
        return self
    def transform(self, X):
        return np.log(X)
#pipeline to compute future return
class Log(BaseEstimator, TransformerMixin):
    def __init__(self): 
        return
    def fit(self, X):
        return self
    def transform(self, X):
        return np.log(X)

In [48]:
feature_list=['DP','PE','BM','CAPE','PCAPrice','BY','DEF','TERM','CAY',
              'SIM',
              'VRP','IC',
              'BDI','NOS','CPI','PCR','MA',
              #'PCA-tech',
              'OIL','SI'] 
R_list=['R_1M','R_3M','R_6M','R_12M']

demedian_list=['VRP','IC']
log_median_list=['BDI']

other_list=[i for i in feature_list if i not in demedian_list+log_median_list]
demedian_pipeline = Pipeline([
        ('selector', DataFrameSelector(demedian_list)),
        ('demedian', DeMedian()),
    ])

log_median_pipeline = Pipeline([
        ('selector', DataFrameSelector(log_median_list)),
        ('log', Log()),
        ('demedian', DeMedian()),
    ])
other_pipeline=Pipeline([
    ('selector', DataFrameSelector(other_list)),
])
full_pipeline = FeatureUnion(transformer_list=[
        ("demedian_pipline", demedian_pipeline),
        ("log_median_pipeline", log_median_pipeline),
        ("other_pipeline", other_pipeline),
        
    ])

In [49]:
train_prepared=full_pipeline.fit_transform(train)

In [50]:
train_prepared

array([[ 0.73708797,         nan, -0.1528084 , ...,  1.        ,
                nan,         nan],
       [-0.85320834,         nan, -0.16382346, ...,  1.        ,
                nan,         nan],
       [-0.0573871 ,         nan, -0.17176631, ...,  1.        ,
                nan,         nan],
       ...,
       [-0.64222115, -4.3       , -0.92404402, ...,  1.        ,
        -0.52399892,  0.98136784],
       [-2.57190703, -6.98      , -0.93083521, ...,  1.        ,
        -0.52399892,  0.53942159],
       [-2.33401527, -5.21      , -0.93083521, ...,  1.        ,
        -0.52399892,  0.48065865]])

# Tables

In [51]:
df_prepared=pd.DataFrame(train_prepared,columns=demedian_list+log_median_list+other_list,index=train.index)

## Table1

In [52]:
df_prepared[R_list]=train[R_list]

In [53]:
summary=df_prepared.describe().T
summary.loc[:,'Skewness']=df_prepared.skew()
summary.loc[:,'Kurtosis']=df_prepared.kurt()
summary.loc[feature_list+R_list]

,count,mean,std,min,25%,50%,75%,max,Skewness,Kurtosis
DP,6275.0,0.030313,0.013598,0.016091,0.021619,0.023294,0.036607,0.077506,1.376078,0.807164
PE,6275.0,17.149349,8.268075,6.632500,13.329156,15.274371,18.615715,75.827909,3.939150,20.689034
BM,3858.0,0.382313,0.081155,0.190303,0.342486,0.367829,0.446324,0.683908,0.091054,0.219331
CAPE,6275.0,18.176844,5.022590,7.395374,14.564245,18.992797,21.157242,30.218934,-0.041247,-0.509246
PCAPrice,6275.0,0.225213,1.545274,-3.531514,-0.769562,-0.478372,1.497023,4.402838,0.823031,-0.244657
BY,6275.0,0.973717,0.100592,0.552394,0.914494,0.973263,1.036844,1.393599,-0.063666,1.270132
DEF,6275.0,0.954183,0.410862,0.550000,0.690000,0.870000,1.060000,3.380000,3.140201,12.986900
TERM,6272.0,1.865515,1.161928,-0.950000,0.910000,2.000000,2.790000,3.870000,-0.290813,-0.984421
CAY,6275.0,0.005230,0.025816,-0.052577,-0.009389,0.004489,0.018684,0.091062,0.601011,1.697840
SIM,6275.0,61.749801,36.172111,0.000000,30.000000,62.000000,93.000000,121.000000,-0.006585,-1.212598


## Table2

In [54]:
df_prepared[feature_list].corr()

,DP,PE,BM,CAPE,PCAPrice,BY,DEF,TERM,CAY,SIM,VRP,IC,BDI,NOS,CPI,PCR,MA,OIL,SI
DP,1.000000,-0.399938,0.713363,-0.881866,0.937773,-0.031397,0.049215,0.240396,-0.036017,-0.042382,0.064234,0.269369,-0.151734,-0.098631,0.371099,-0.623930,-0.012204,-0.289627,0.142280
PE,-0.399938,1.000000,0.087505,0.260784,-0.629854,0.138330,0.110652,0.281411,0.440635,-0.012828,0.119581,0.020482,0.151162,-0.319521,-0.383968,0.280559,0.042150,-0.021459,-0.133759
BM,0.713363,0.087505,1.000000,-0.771820,0.217037,-0.283958,0.600965,0.448422,0.752290,-0.079554,0.309994,0.705814,-0.074796,-0.136490,-0.365407,-0.336642,-0.041639,-0.320335,0.059417
CAPE,-0.881866,0.260784,-0.771820,1.000000,-0.792653,0.122016,-0.210838,-0.449741,-0.333042,0.023355,-0.074996,-0.485870,0.008292,0.138378,-0.230004,0.776953,0.095059,0.173875,-0.016728
PCAPrice,0.937773,-0.629854,0.217037,-0.792653,1.000000,-0.043157,-0.117171,0.076835,-0.241570,-0.021724,0.003178,0.084688,-0.202552,-0.022177,0.418389,-0.564653,0.029004,-0.264859,0.060071
BY,-0.031397,0.138330,-0.283958,0.122016,-0.043157,1.000000,-0.329162,0.092459,-0.018819,0.140167,-0.181390,-0.478965,0.106304,0.090832,0.050213,0.028541,0.255187,0.099025,-0.072033
DEF,0.049215,0.110652,0.600965,-0.210838,-0.117171,-0.329162,1.000000,0.253921,0.500169,-0.020849,0.246082,0.330308,0.055089,-0.249626,-0.243199,-0.172638,-0.445411,0.047364,-0.029534
TERM,0.240396,0.281411,0.448422,-0.449741,0.076835,0.092459,0.253921,1.000000,0.542635,0.061815,0.092356,0.398803,-0.071969,-0.238081,-0.238530,-0.238590,-0.077520,-0.231822,-0.053362
CAY,-0.036017,0.440635,0.752290,-0.333042,-0.241570,-0.018819,0.500169,0.542635,1.000000,0.127366,0.075844,0.341942,0.302608,-0.161129,-0.324996,-0.313343,-0.098928,0.013124,-0.069644
SIM,-0.042382,-0.012828,-0.079554,0.023355,-0.021724,0.140167,-0.020849,0.061815,0.127366,1.000000,-0.152841,-0.139909,-0.022382,-0.048267,-0.026416,0.043539,0.098697,-0.009164,-0.089550


### Table 3

In [55]:
df_prepared.corr().loc[feature_list,R_list]

,R_1M,R_3M,R_6M,R_12M
DP,0.062153,0.129363,0.218757,0.281471
PE,-0.026175,-0.045724,-0.099814,-0.082092
BM,0.181451,0.284133,0.434788,0.588675
CAPE,-0.084414,-0.153524,-0.251804,-0.369015
PCAPrice,0.069317,0.139280,0.228652,0.294679
BY,-0.048347,-0.043957,-0.044397,0.018440
DEF,-0.041985,-0.066419,-0.008650,0.045173
TERM,-0.030163,-0.056934,-0.041130,0.062733
CAY,0.037669,0.059267,0.081756,0.165351
SIM,-0.033490,-0.123173,-0.149468,-0.009074
